## Assignment

A retailer called SpreeMart wants to improve their service experience for their customers which have different needs from their different profiles. 

As a data scientist at SpreeMart, you are asked to develop a model in Python to divide broad customers into sub-groups of customers based on some type of shared characteristics and extract useful insights from the data and your model with a given sample of customer data in the files attached as following. 

- sm_customers.csv describes the customer's information and its location.
- sm_items.csv includes data of items purchased within each order. 
- sm_orders.csv This file shows data of each order. 
- sm_payments.csv file details the order payment options. 

SpreeMart expects you to do good practice for data science including data exploratory, model development, insight interpretation and in addition, sharing the results with stakeholders by preparing a presentation for an audience of mixed technical abilities on your results and thinking process.

---

## Solution overview

SpreeMart is seeking to enhance its customer service experience by better understanding the diverse needs of its customers. Our objective is to analyze customer data and develop a model that segments customers into distinct groups based on shared characteristics. This segmentation will enable SpreeMart to tailor their services more effectively to each group.

### Expactation and solution approach

#### 1. Data exploration: 

- **Data cleaning and preparation:** We'll start with loading and examinating the data. We'll identify and handle missing values, and perform any necessary data transformations.
- **Exploratory Data Analysis (EDA):** we'll loop deep into the data, looking at distributions, patterns, and relationships. This will help us understand the data's nature and modeling approach.

#### 2. Model development:
- **Customer Segmentation Model:** We will develop clustering techniques to segment the customers. The choice of technique will depend on the data's characteristics and the insights we wish to derive.

#### 3. Insight Generation:
- **Analysis and Insight Generation:** we'll analyze each segment to disciver unique traits and behaviors. This will lead to actionable insights which can use to enhance customer experience and service personalization.

#### 4. Presentation of Results:
- **Presentation and Reporting:** We'll prepare a presentation to summarize our findings, and recommendations. This presentation will be tailored for various level of audiences to communicate our insights and implications business strategy.

---

### 1. Data exploration

In [23]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

customers = pd.read_csv('./data/sm_customers.csv')
items = pd.read_csv('./data/sm_items.csv')
orders = pd.read_csv('./data/sm_orders.csv')
payments = pd.read_csv('./data/sm_payments.csv')


##### 1.1 Basic profiling

In [38]:
# Example data
print(customers.head())
print(items.head())
print(orders.head())
print(payments.head())

                        customer_id                customer_unique_id  \
0  06b8999e2fba1a1fbc88172c00ba8bc7  861eff4711a542e4b93843c6dd7febb0   
1  18955e83d337fd6b2def6b18a428ac77  290c77bc529b7ac935b93aa66c333dc3   
2  4e7b3e00288586ebd08712fdd0374a03  060e732b5b29e8181a18229c7b0b2b5e   
3  b2b6027bc5c5109e529d4dc6358b12c3  259dac757896d24d7702b9acbbff3f3c   
4  4f2d8ab171c80ec8364f7c12e35b23ad  345ecd01c38d18a9036ed96c73b8d066   

         customer_city customer_state  customer_zip_code  
0       New Jamesville             AK              99765  
1           West Peter             VA              23267  
2            New Karen             OR              97707  
3  North Michelleburgh             VA              22322  
4      East Jasonhaven             NM              88080  
                           order_id  order_item_id  \
0  00010242fe8c5a6d1ba2dd792cb16214              1   
1  00018f77f2f0320c557190d7a144bdd3              1   
2  000229ec398224ef6ca0657da4fc703e          

In [25]:
# Loop display summary of each DataFrame

dict = {'customers': customers, 'items': items, 'orders': orders, 'payments': payments}

for key, df in dict.items():
    print(f"=========== Info: {key} ===========")
    print(df.info())
    print("\n")

    # Identify missing values
    print(f"--- Missing Values: {key} ---")
    print(df.isnull().sum())

    # Identify duplicate rows
    print(f"--- Duplicate Rows: {key} ---")
    print(df.duplicated().sum())

    # Identify unique values
    print(f"--- Unique Values: {key} ---")
    print(df.nunique())

    print("\n")




=========== Info: customers ===========
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   customer_id         99441 non-null  object
 1   customer_unique_id  99441 non-null  object
 2   customer_city       99441 non-null  object
 3   customer_state      99441 non-null  object
 4   customer_zip_code   99441 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 3.8+ MB
None


--- Missing Values: customers ---
customer_id           0
customer_unique_id    0
customer_city         0
customer_state        0
customer_zip_code     0
dtype: int64
--- Duplicate Rows: customers ---
0
--- Unique Values: customers ---
customer_id           99441
customer_unique_id    96096
customer_city         38038
customer_state           51
customer_zip_code     50545
dtype: int64


=========== Info: items ===========
<class 'pandas.core.frame.DataFr

----

##### 1.2 Customer investigstion

We found that customer table has customer_id and customer_unique_id. We want to their relationship and which one is more useful for our analysis.


In [37]:

# total number of rows
print("Total customer rows count:", customers.shape[0])

# Check unique counts
print("Unique customer_id count:", customers['customer_id'].nunique())
print("Unique customer_unique_id count:", customers['customer_unique_id'].nunique())
print("\n")

# Check if there are multiple customer_ids for a single customer_unique_id
customer_id_counts = customers.groupby('customer_unique_id')['customer_id'].nunique()

# Customer_id counts per customer_unique_id
print("Distribution of customer_id counts per customer_unique_id:")
print(customer_id_counts.value_counts())
print("\n")

# Analyzing any customer_unique_id with more than one customer_id
multi_customer_ids = customer_id_counts[customer_id_counts > 1]

# rowcount multi_customer_ids
print("Count number of customer_unique_ids which have multiple customer_id:", multi_customer_ids.count())
print("Examples:")
print(multi_customer_ids.tail(5))


Total customer rows count: 99441
Unique customer_id count: 99441
Unique customer_unique_id count: 96096


Distribution of customer_id counts per customer_unique_id:
customer_id
1     93099
2      2745
3       203
4        30
5         8
6         6
7         3
9         1
17        1
Name: count, dtype: int64


Count number of customer_unique_ids which have multiple customer_id: 2997
Examples:
customer_unique_id
ff36be26206fffe1eb37afd54c70e18b    3
ff44401d0d8f5b9c54a47374eb48c1b8    2
ff8892f7c26aa0446da53d01b18df463    2
ff922bdd6bafcdf99cb90d7f39cea5b3    3
ffe254cc039740e17dd15a5305035928    2
Name: customer_id, dtype: int64


##### Customer_id and customer_unique_id interpretation

- **customer_id** seems to be an order-specific identifier, unique to each transaction. We can use this ID when analyzing at an order level
- **customer_unique_id** appears to be a unique identifier for each **individual customer** across multiple transactions. This ID is crucial for customer-level analysis, especially for understanding customer behavior over time

**Assumption:** 

- We will use **customer_unique_id** for our analysis, as it will allow us to group customers based on their overall behavior 

----

##### 1.3 Data cleaning and preparation

In order to effectively segment customers we need to start from consolidating customer data from different tables and then perform data cleaning and prepate data for feature engineering.

**1.3.1 Consolidate Customer Data**

In [50]:
# Join customers with orders
combined_data = pd.merge(orders, customers, on='customer_id', how='left')

# Join with items
combined_data = pd.merge(combined_data, items, on='order_id', how='left')

# Join with payments
combined_data = pd.merge(combined_data, payments, on='order_id', how='left')

print(f"--- Info: Consolidated data for customer segmantation ---")
print(combined_data.info())
print("\n")

# Check for missing values
print(f"--- Missing values ---")
print(combined_data.isnull().sum())
print("\n")


--- Info: Consolidated data for customer segmantation ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118434 entries, 0 to 118433
Data columns (total 22 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   order_id                       118434 non-null  object 
 1   customer_id                    118434 non-null  object 
 2   order_status                   118434 non-null  object 
 3   order_purchase_timestamp       118434 non-null  object 
 4   order_approved_at              118258 non-null  object 
 5   order_delivered_carrier_date   116360 non-null  object 
 6   order_delivered_customer_date  115037 non-null  object 
 7   order_estimated_delivery_date  118434 non-null  object 
 8   customer_unique_id             118434 non-null  object 
 9   customer_city                  118434 non-null  object 
 10  customer_state                 118434 non-null  object 
 11  customer_zip_code              11

**1.3.1 Prepare Data**

1. **Remove Missing Values**: Rows with missing values will be removed.
2. **Convert Data Types**: Convert date columns from object to datetime format.

In [54]:
consolidated_data = combined_data

# 1. Removing Missing Values
consolidated_data.dropna(inplace=True)

# 2. Converting Data Types
date_columns = ['order_purchase_timestamp', 'order_approved_at', 
                'order_delivered_carrier_date', 'order_delivered_customer_date', 
                'order_estimated_delivery_date', 'shipping_limit_date']

for col in date_columns:
    consolidated_data[col] = pd.to_datetime(consolidated_data[col], errors='coerce')


print(consolidated_data.tail(5))

                                order_id                       customer_id  \
118429  63943bddc261676b46f01ca7ac2f7bd8  1fca14ff2861355f6e5f14306ff977a7   
118430  83c1379a015df1e13d02aae0204711ab  1aa71eb042121263aafbe80c1b562c9c   
118431  11c177c8e97725db2631073c19f07b62  b331b74b18dc79bcdf6532d51e1637c1   
118432  11c177c8e97725db2631073c19f07b62  b331b74b18dc79bcdf6532d51e1637c1   
118433  66dea50a8b16d9b4dee7af250b4be1a5  edb027a75a1449115f6b43211ae02a24   

       order_status order_purchase_timestamp   order_approved_at  \
118429    delivered      2021-02-06 12:58:58 2021-02-06 13:10:37   
118430    delivered      2020-08-27 14:46:43 2020-08-27 15:04:16   
118431    delivered      2021-01-08 21:28:27 2021-01-08 21:36:21   
118432    delivered      2021-01-08 21:28:27 2021-01-08 21:36:21   
118433    delivered      2021-03-08 20:57:30 2021-03-09 11:20:28   

       order_delivered_carrier_date order_delivered_customer_date  \
118429          2021-02-07 23:22:42           2021-02

----

Feature Engineering